# This notebook provides the commands needed to read in an NWB file with raw data taken from a trodes .rec file and to copy the relevant metadata into a DataJoint Schema designed to mimic NWB structure

## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

In [1]:
cd ../schema

/Users/loren/Src/NWB/nwb_datajoint/schema


#### Load all of the relevant modules

In [2]:
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj
import common_session
import common_ephys
import common_device
import common_lab
import common_task
import common_behav
import common_interval
import common_region
import common_filter
import nwb_dj

#franklabnwb functions
#from franklabnwb import fl_ns_path
#import franklabnwb.nspike_helpers as ns
#import franklabnwb.query_helpers as qu
#import franklabnwb.fl_extension as fle
#import franklabnwb.fl_extension as flh

Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


#### Next we specify the NWB file name and populate the rest of the schema
#### Replace the code below with the text for the path to the NWB file

In [3]:
nwb_raw_file_name = '/data/nwb_builder_test_data/beans20190718.nwb'

### temporary temporary file; FIX when NWB file handing sorted out

In [4]:
nwb_file_name = '/data/nwb_builder_test_data/beans20190718_pp.nwb'

In [15]:
nwb_dj.NWBPopulate([nwb_raw_file_name]) 

Importing NWB file /data/nwb_builder_test_data/beans20190718.nwb
No Apparatus module found in /data/nwb_builder_test_data/beans20190718_pp.nwb

Importing raw data: Estimated sampling rate:	20000.0 Hz
                    Number of valid intervals:	5
Processing raw position data. Estimated sampling rate: 30.0 Hz


#### The code below creates a diagram of the various schema and their relationships

In [17]:
a = common_session.LinkedNwbfile().create(nwb_file_name)

writing new NWB file /data/nwb_builder_test_data/beans20190718_pp00000000.nwb
inserted file


In [18]:
import numpy as np
with pynwb.NWBHDF5IO(a, "a") as io:
    nwbf= io.read()
nwbf
electrode_table_region = nwbf.create_electrode_table_region([0,2], 'filtered electrode table')
es = pynwb.ecephys.ElectricalSeries(name='filt test', data=np.empty((2,2), dtype='i2'),
                                    electrodes=electrode_table_region, timestamps= np.empty(0))
test = np.ones((10,1))
nwbf.add_scratch(test, "test data", notes="test")
#nwbf.add_analysis(es)
with pynwb.NWBHDF5IO(nwb_file_name, "a") as io:
    io.write(nwbf)

io.close()

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /acquisition/e-series
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/devices/header_device
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/devices/probe 0
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/extracellular_ephys/0
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/extracellular_ephys/1
  wa

Exception: Could not construct DynamicTable object due to: 'x'

In [15]:
common_filter.create_standard_filters()
common_ephys.LFPElectrode().set_lfp_elect([1, 2, 3])
common_ephys.LFP.populate()

writing new NWB file /data/nwb_builder_test_data/beans20190718_pp00000000.nwb
inserted file
*nwb_file_name *linked_file_n
+------------+ +------------+
/data/nwb_buil /data/nwb_buil
 (Total: 1)

output file name /data/nwb_builder_test_data/beans20190718_pp00000000.nwb
First ind 2133, last ind 21841133, diff 21839000, total size 21843266
Output array should have shape (1091950, 3) and dtype <f8
dtype = <f8, <HDF5 dataset "data": shape (138007758, 256), type "<i2">
First ind 2133, last ind 4258364, diff 4256231, total size 4260497
Output array should have shape (212812, 3) and dtype <f8
dtype = <f8, <HDF5 dataset "data": shape (138007758, 256), type "<i2">
First ind 2133, last ind 51156998, diff 51154865, total size 51159131
Output array should have shape (2557744, 3) and dtype <f8
dtype = <f8, <HDF5 dataset "data": shape (138007758, 256), type "<i2">
First ind 2133, last ind 27140904, diff 27138771, total size 27143037
Output array should have shape (1356939, 3) and dtype <f8
dtype = <f

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /acquisition/e-series
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/devices/header_device
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/devices/probe 0
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/extracellular_ephys/0
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/extracellular_ephys/1
  wa

Exception: Could not construct DynamicTable object due to: 'x'

In [8]:
common_session.Nwbfile().delete()

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


About to delete:
`common_ephys`.`_electrode_config__electrode`: 134 items
`common_ephys`.`_electrode_config__electrode_group`: 2 items
`common_ephys`.`_electrode_config`: 1 items
`common_behav`.`_raw_position`: 1 items
`common_ephys`.`_raw`: 1 items
`common_interval`.`interval_list`: 2 items
`common_session`.`_experimenter_list__experimenter`: 1 items
`common_session`.`_experimenter_list`: 1 items
`common_session`.`_session`: 1 items
`common_session`.`nwbfile`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [ ]:
common_session.LinkedNwbfile()

In [ ]:
interval_list = (common_interval.IntervalList() & {'nwb_file_name': nwb_file_name ,
                                                           'interval_name': 'raw data valid times'}).fetch1('valid_times')

In [10]:
common_session.Nwbfile().delete()

About to delete:
`common_session`.`nwbfile`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [22]:
common_filter.filter_data_nwb(nwb_file_name, timestamps, data, fs_orig, filter_b, interval_list, electrodes, decimation):


In [6]:
#### And now that we have the data in DataJoint, we can do queries to find time intervals we might be interested in
#### In the example below, we get all of the intervals where the task_name was sleep.

In [7]:
common_device.Probe.drop()

`common_device`.`probe` (0 tuples)
`common_device`.`probe__shank` (0 tuples)
`common_device`.`probe__electrode` (0 tuples)
`common_ephys`.`_electrode_config__electrode_group` (0 tuples)
`common_ephys`.`_units` (0 tuples)
`common_ephys`.`_electrode_config__electrode` (0 tuples)
`common_ephys`.`l_f_p_electrode` (0 tuples)
`common_ephys`.`__l_f_p` (0 tuples)
`common_ephys`.`l_f_p_band_parameters` (0 tuples)
`common_ephys`.`__l_f_p_band` (0 tuples)
`common_ephys`.`__decomp_series` (0 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [26]:
sleep_intervals['valid_times']

array([array([1.13614661e+09, 1.13614794e+09]),
       array([1.13614908e+09, 1.13615003e+09]),
       array([1.13615118e+09, 1.13615241e+09]),
       array([1.13615411e+09, 1.13615495e+09])], dtype=object)

In [3]:
common_ephys

NameError: name 'common_ephys' is not defined

In [1]:
common_session.Nwbfile.drop()

NameError: name 'common_session' is not defined

In [21]:
common_device.Probe().Electrode()

probe_type,probe_shank shank number within probe,probe_electrode electrode,contact_size (um) contact size,rel_x (um) x coordinate of the electrode within the probe,rel_y (um) y coordinate of the electrode within the probe,rel_z (um) z coordinate of the electrode within the probe
128c-4s8mm6cm-20um-40um-sl,0,0,20.0,0.0,0.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,1,20.0,0.0,-40.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,2,20.0,0.0,-80.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,3,20.0,0.0,-120.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,4,20.0,0.0,-160.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,5,20.0,0.0,-200.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,6,20.0,0.0,-240.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,7,20.0,0.0,-280.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,8,20.0,0.0,-320.0,0.0
128c-4s8mm6cm-20um-40um-sl,0,9,20.0,0.0,-360.0,0.0


In [10]:
common_ephys

In [5]:
a = common_ephys.SpikeSortingGroup().set_by_electrode_group(nwb_file_name)

In [12]:
rx, ry = (common_device.Probe.Electrode() & {'probe_electrode' : 0}).fetch('rel_x','rel_y')

In [20]:
common_ephys.SpikeSortingGroup().write_prb(nwb_file_name, '/data/nwb_builder_test_data/test.prb')

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions

{0: {'channels': [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31], 'geometry': [[0.0, 0.0], [0.0, -40.0], [0.0, -80.0], [0.0, -120.0], [0.0, -160.0], [0.0, -200.0], [0.0, -240.0], [0.0, -280.0], [0.0, -360.0], [0.0, -440.0], [0.0, -520.0], [0.0, -600.0], [0.0, -680.0], [0.0, -760.0], [0.0, -840.0], [0.0, -920.0], [0.0, -1000.0], [0.0, -1080.0], [0.0, -1160.0], [0.0, -1240.0]], 'label': ['0', '1', '2', '3', '4', '5', '6', '7', '9', '11', '13', '15', '17', '19', '21', '23', '25', '27', '29', '31']}, 1: {'channels': [33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63], 'geometry': [[345.0, -40.0], [345.0, -120.0], [345.0, -200.0], [345.0, -280.0], [345.0, -360.0], [345.0, -440.0], [345.0, -520.0], [345.0, -600.0], [345.0, -680.0], [345.0, -760.0], [345.0, -840.0], [345.0, -920.0], [345.0, -1000.0], [345.0, -1080.0], [345.0, -1160.0], [345.0, -1240.0]], 'label': ['33', '35', '37', '39', '41', '43', '45', '47', '49', '51', '53', '55', '57', '59', '61', '63

In [16]:
import os
import pynwb
import ndx_fl_novela.probe

nwb_file_name = '/data/nwb_builder_test_data/beans20190718s1.nwb'

in_io  = pynwb.NWBHDF5IO(nwb_file_name, 'r')
nwbf_in = in_io.read()

nwbf_out = nwbf_in.copy()


n_linked_files = 4
# name the file, adding the number of links with preceeding zeros
nwb_out_file_name = os.path.splitext(nwb_file_name)[0] + str(n_linked_files).zfill(5) + '.nwb'

# write the linked file
print(f'writing new NWB file {nwb_out_file_name}')
with pynwb.NWBHDF5IO(nwb_out_file_name, mode='a', manager=in_io.manager) as io:
    io.write(nwbf_out)
    
io

in_io.close()
# test append to file - this works
new_io = pynwb.NWBHDF5IO(nwb_out_file_name, 'a')
nwbf_new = new_io.read()
print(f'test append to created file: {nwbf_new}')
new_io.write(nwbf_new)
new_io.close()

writing new NWB file /data/nwb_builder_test_data/beans20190718s100004.nwb


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/hdmf/backends/hdf5/h5tools.py:436: BrokenLinkWarning: /general/devices/probe 0
  warnings.warn(os.path.join(h5obj.name, k), BrokenLinkWarning)


test append to created file: root pynwb.file.NWBFile at 0x4944538192
Fields:
  acquisition: {
    e-series <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    header_device <class 'abc.HeaderDevice'>
  }
  electrode_groups: {
    electrode group 0 <class 'abc.NwbElectrodeGroup'>,
    electrode group 1 <class 'abc.NwbElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  epochs: epochs <class 'pynwb.epoch.TimeIntervals'>
  experiment_description: Reinforcement learning
  experimenter: ['Alison Comrie']
  file_create_date: [datetime.datetime(2020, 5, 8, 18, 48, 20, 634706, tzinfo=tzoffset(None, -25200))]
  identifier: 293c8f78-9197-11ea-b128-88e9fe59b5c6
  institution: University of California, San Francisco
  intervals: {
    epochs <class 'pynwb.epoch.TimeIntervals'>
  }
  lab: Loren Frank
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  session_description: Reinforcement leaarning
  session_start_time: 2019-07-18 15:2

In [11]:
electrodes = (common_ephys.ElectrodeConfig.Electrode() & {'nwb_file_name' : nwb_file_name} & {'bad_channel' : 'False'}).fetch()

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions

In [13]:
electrodes

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions

array([('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   0, '128c-4s8mm6cm-20um-40um-sl', 0,   0, 2, '0', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
       ('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   1, '128c-4s8mm6cm-20um-40um-sl', 0,   1, 2, '1', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
       ('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   2, '128c-4s8mm6cm-20um-40um-sl', 0,   2, 2, '2', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
       ('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   3, '128c-4s8mm6cm-20um-40um-sl', 0,   3, 2, '3', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
       ('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   4, '128c-4s8mm6cm-20um-40um-sl', 0,   4, 2, '4', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
       ('/data/nwb_builder_test_data/beans20190718_pp.nwb', '0',   5, '128c-4s8mm6cm-20um-40um-sl', 0,   5, 2, '5', 0., 0., 0., 'None', 0., 'False', 0., 0., 0., ''),
    

In [14]:
elect_info = (electrodes['electrode_group_name'], electrodes['probe_shank'])

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions

In [17]:
import numpy as np
egroups = np.unique(electrodes['electrode_group_name'])


In [18]:
egroups

array(['0', '1'], dtype=object)

In [20]:
e = egroups[0]

In [25]:
shanks = electrodes['probe_shank'][electrodes['electrode_group_name'] == e]

In [31]:
shank_dict = dict()
# for each electrode group, get a list of the unique shanks
shank_dict[e] = np.unique(electrodes['probe_shank'][electrodes['electrode_group_name'] == e])

In [32]:
shank_dict[e]

array([0, 1, 2, 3])

In [50]:
sort_group = 0
elect_dict = dict()
elect_dict['nwb_file_name'] = nwb_file_name
for e_group in egroups:
    elect_dict['electrode_group_name'] = e_group
    # for each electrode group, get a list of the unique shank numbers
    shank_list = np.unique(electrodes['probe_shank'][electrodes['electrode_group_name'] == e_group])
    # get the indices of all electrodes in for this group / shank and set their sorting group
    for shank in shank_list:
        shank_elect = electrodes['electrode_id'][np.logical_and(electrodes['electrode_group_name'] == e_group,
                                                                electrodes['probe_shank'] == shank)]
    # create a 2d array for these electrodes
    sort_insert = list()
    for elect in shank_elect:
        sort_insert.append([nwb_file_name, e_group, elect, sort_group])

[autoreload of common_ephys failed: Traceback (most recent call last):
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/IPython/extensions

In [51]:
sort_insert

[['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 225, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 227, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 229, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 231, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 233, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 235, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 237, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 239, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 241, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 243, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 245, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 247, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 249, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.nwb', '1', 251, 0],
 ['/data/nwb_builder_test_data/beans20190718_pp.

In [47]:
a

[1, '/data/nwb_builder_test_data/beans20190718_pp.nwb']